In [13]:
import pickle
mem = pickle.load(open('first.dump', 'rb'))

In [9]:
len(mem)

958

In [10]:
index = 0
for m in mem:
    print(m)
    index += 1
    if index >= 10:
        break

('0x7f2ea37da000', 12288)
('0x7f2ea20dc000', 4096)
('0x7f2e90931000', 114688)
('0x7f2ea6471000', 4096)
('0x7f2ea6de7000', 20480)
('0x7f2ea6459000', 4096)
('0x7f2e88047000', 66818048)
('0x7f2ea5f34000', 45056)
('0x7f2ea0b8a000', 4096)
('0x7f2ea0bcd000', 8192)


In [15]:
delta = pickle.load(open('delta.dump', 'rb'))
print(len(delta))

0
